In [10]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import torch
import sys
import kaolin as kal
import matplotlib.pyplot as plt
import igl
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

# obj_path = 'data/demo/penguin.obj'
obj_path = 'data/FAUST/scans/tr_scan_000.obj'
# obj_path = 'data/demo/B075QFCHM9.obj'

Torch will run on: cuda:0


In [11]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)
# face_areas = kal.ops.mesh.face_areas(
#     vertices_tensor.unsqueeze(0),
#     faces_tensor
# ).view(len(mesh.faces))

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  10061
Number of faces:  19999


In [12]:
# Visualize mesh
mp.plot(vertices, faces)
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0275495…

In [13]:
adj = igl.adjacency_matrix(faces)
cc = igl.connected_components(adj)
print('Number of connected components:', cc[0])
mp.plot(vertices, faces, cc[1])
plt.show()

Number of connected components: 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0275495…